# Projet IF29 P22
### Détection des influenceurs sur Twitter en appliquant les méthodes non supervisée et supervisée

In [ ]:
# Créer la BD en utilisant le fichier 'Worldcup 200Tweets' sur moodle
from pymongo import MongoClient
import os
import json

client = MongoClient('localhost', 27017)
db = client['______'] # nom de BD
collection = db['______'] #nom de collection
parent_dir = ("_____________________________________") #adresse de fichier
list_filename = os.listdir(parent_dir)

for filename in list_filename:
    filepath = parent_dir + filename
    with open(filepath, encoding='utf-8') as f:
        lines = f.readlines()
        f.close()
    for line in lines:
        document = json.loads(line[:-1])
        collection.insert_one(document)

In [1]:
# Connecter à la BD
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['if29_projet']
collection = db['if29_projet']

In [2]:
# Créer le dataframe en utilisant les attributs qui nous intéressent concernant les utilisateurs
import numpy as np
import pandas as pd

cursor = collection.aggregate(
    [
        {"$group" : {
            "_id" : "$user.id", 
            "friends_count" : {"$last" : "$user.friends_count"},
            "favourites_count" : {"$last" : "$user.favourites_count"},
            "statuses_count" : {"$last" : "$user.statuses_count"},
            "followers_count" : {"$last" : "$user.followers_count"},
             "created_time" : {"$last" : "$user.created_at"}
        }
        }
    ]
)
df = pd.json_normalize(cursor)

In [3]:
# Première analyse sur le résumé des données quantitatives
df.iloc[:,[1,2,3,4]].describe().astype(int)

,friends_count,favourites_count,statuses_count,followers_count
count,267468,267468,267468,267468
mean,1101,9108,22204,7829
std,5539,25271,54745,181738
min,0,0,1,0
25%,186,380,1329,124
50%,408,1946,6275,363
75%,906,7547,21751,1019
max,544830,1718515,8898917,30755372


In [4]:
# Calculer les indicateurs (attributs dérivés) ci-dessous destinés à la description des 'influenceurs'
# 1) Visibilité: Vis
# 2) Nombre moyen de 'aimer' par tweet: avg_fav 
# 3) Ratio entre nombre de 'abonné' et nombre de 'abonnement': r_fri_follow
# 4) la fréquence de publication de tweets depuis la création de compte jusqu'à t0 = 01/01/2019: frequency

In [4]:
# 1) Visibilité: Vis

# Regrouper les textes de tweet par chaque utilisateur
cursor1 = collection.aggregate(
    [        
         {"$group" : {
            "_id" : "$user.id", 
            
            "tweets" : {"$push": "$text"}
        }
        }
    ]
)
df1 = pd.json_normalize(cursor1)

def calcul_visibility(tweets):
    s = 0
    for tweet in tweets:
        s += tweet.count("@")*11.4  # le coût moyen pour @ est 11,4
        s += tweet.count("#")*11.6  # le coût moyen pour # est 11,6 cf.fichier 'SPOT'
    return s/(140*len(tweets))

visibilities = []
for texts in df1.iloc[:,1]:
    visibilities.append(calcul_visibility(texts)) # mettre les résultats dans une liste

In [5]:
# 2) Nombre moyen de 'aimer' par tweet: avg_fav
avg_fav = df.favourites_count/df.statuses_count

In [6]:
# 3) Ratio entre nombre de 'abonné' et nombre de 'abonnement': r_fri_follow
r_fri_follow = df.followers_count/df.friends_count

In [7]:
# 4) la fréquence de publication de tweets depuis la création de compte jusqu'à t0 = 01/01/2019: frequency
projection = { 
    'user.statuses_count': 1,
    'user.created_at': 1,
    '_id': 0}
cursor3 = collection.find({},projection)

import time
def ratio(n_tweets, date):
    time_array = time.strptime(date, "%a %b %d %H:%M:%S +0000 %Y")
    time_stamp = time.mktime(time_array)
    time_stamp_0 = time.mktime(time.strptime("Mon Jan 1 00:00:00 2019", '%a %b %d %H:%M:%S %Y'))
    return n_tweets/(time_stamp_0 - time_stamp)

frequency = []
for index, row in df.iterrows():
    n_tweet = row["statuses_count"]
    created_date = row["created_time"]
    frequency.append(ratio(n_tweet, created_date)*100) # on multiplie une constante = 100 pour amplifier les résultats 

In [8]:
# Générer un dataframe en regroupant les quatres indicateurs calculés
features = pd.DataFrame()
features["user.id"] = df.iloc[:,[0]]
features["vis"] = visibilities
features["r_fri_follow"] = r_fri_follow
features["avg_fav"] = avg_fav
features["frequency"] = frequency

In [11]:
# On supprime toutes les lignes qui contiennent 'Nan' ou 'inf'
features.dropna(inplace=True)

In [ ]:
# On normalise les données pour effectuer les classifications
from sklearn import preprocessing

features_normalized = preprocessing.normalize(features.iloc[:,[1,2,3,4]])

# Vérification
print("La moyenne : ",np.mean(features_normalized,axis=0))
print("L'écart type : ",np.std(features_normalized,axis=0))

In [ ]:
# On effectue l'ACP pour réduire les 4 dimensions
from sklearn.decomposition import PCA

acp = PCA(svd_solver='auto')
coord = acp.fit_transform(features_normalized) 
acp.explained_variance_ratio_ # les pourcentage de variance projectée sur chaque axe (composante principale)
coord # les coordonées des individus dans les nouveaux repères

In [ ]:
# Basé sur le résultat d'ACP, on effectue le K-means dans le plan (1,2) qui se compose de deux composantes dont inerties les plus élevées 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

kmeans = KMeans(n_clusters=2, random_state=0) # Selon le sujet, ici on ne choisit que de classer en 2 clusters.
labels = kmeans.fit_predict(coord[:,[0,1]]) # on ne prend que les coordonées de l'axe 1 et 2

np.unique(labels, return_counts=True) # afficher la condition de séparation

labels0 = coord[labels == 0]
labels1 = coord[labels == 1]
plt.scatter(np.random.choice(labels0[:,0],100), np.random.choice(labels0[:,1],100), c='red') # Pour voir plus clairement, on prend aléatoirement 100 points pour tracer le graphique
plt.scatter(np.random.choice(labels1[:,0],100), np.random.choice(labels1[:,1],100), c='blue') 


In [ ]:
# Avant de passer à l'approche supervisée, il faut d'abord labeliser les données
# Dans cette étape on va utiliser un échantillon de taille 50000 pour entraîner le modèle
# Comme on a 4 indicateurs, on décide de les combiner en utilisant la méthode d'entropie

In [ ]:
# Méthode d'entropie

data = features.iloc[:,[1,2,3,4]]
indicateur = data.columns.tolist()   ## nombre des indicateurs
profil = data.index.tolist()    ## nombre des profils
value = data.values

# définir la fonction de normalisation, on rajoute une constante 0,01
def std_data(value,flag):
    for i in range(len(indicateur)):
        if flag[i]=='+':
            value[:,i]=(value[:,i]-np.min(value[:,i],axis=0))/(np.max(value[:,i],axis=0)-np.min(value[:,i],axis=0))+0.01
        elif flag[i]=='-':
            value[:,i]=(np.max(value[:,i],axis=0)-value[:,i])/(np.max(value[:,i],axis=0)-np.min(value[:,i],axis=0))+0.01
    return value

# définir la fonction d'entropie et calculer les poids et les entropies
def cal_weight(indicateur,profil,value):
    p= np.array([[0.0 for i in range(len(indicateur))] for i in range(len(profil))])                    
    for i in range(len(indicateur)):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
        
    e=-1/np.log(len(profil))*sum(p*np.log(p))      #calcul d'entropie
    g=1-e     
    w=g/sum(g)     #calcul des poids
    return w

# normalisation des données
flag=["+","+","+","+"]  ## '+' ou '-' désigne le sens positif ou négatif des indicateurs
std_value=std_data(value,flag)
std_value.round(3)

# Résultat
w=cal_weight(indicateur,profil,std_value)
w=pd.DataFrame(w,index=data.columns,columns=['Poids'])
print("#######Poids:#######")
print(w)
score=np.dot(std_value,w).round(2)
score=pd.DataFrame(score,index=data.index,columns=['Notes']).sort_values(by =['Notes'],ascending = False)

# la note de chaque utilisateur représente la correspondance avec les profils qu'on va détecter
# Si une note plus élevée, le profil correspondant est plus probablement censé un influnceur

In [ ]:
# A l'aide de résultat ci-dessus, on peut labeliser les profils 